
Model
---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.0 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
     |████████████████████████████████| 101 kB 13.9 MB/s 
     |████████████████████████████████| 596 kB 75.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.1 MB/s 


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
del df['Review']
del df['giai_tri']
del df['luu_tru']
del df['nha_hang']
del df['an_uong']
del df['di_chuyen']
del df['mua_sam']

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 6.8 MB/s 
     |████████████████████████████████| 212 kB 61.9 MB/s 
     |████████████████████████████████| 141 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 47.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from datasets import *


Test model
---



In [ ]:
df

,Unnamed: 0,clean_review,labels
0,0,bánh rất nhiều tôm to tôm giòn nằm chễm_chệ tr...,{an_uong#very_postive}
1,1,đi quy nhơn cứ lo không có pizza ăn nhưng nhờ ...,{an_uong#very_postive}
2,2,các món ăn_ở đây không có gì đặc_sắc có_lẽ nấu...,"{nha_hang#very_negative},{an_uong#very_negative}"
3,3,lần thứ 2 quay lại đây vì giới_thiệu quán cà_p...,{an_uong#negative}
4,4,rất khuyên bạn nên ghé thăm quán bar thể_thao này,{giai_tri#normal}
...,...,...,...
2759,2825,hải_sản chủ_yếu buổi chiều giá cao hơn chợ đầm,{mua_sam#postive}
2760,2826,ngoài_ra khách_sạn ở khu trung_tâm gần bãi biể...,{luu_tru#very_postive}
2761,2827,khách_sạn sạch_sẽ tiện_nghi dịch_vụ đa_dạng,{luu_tru#very_postive}
2762,2828,sẽ ủng_hộ lâu_dài tuy_nhiên còn vài điều cần g...,{di_chuyen#negative}


In [ ]:
!cp -r /content/drive/MyDrive/t5-small-vietnamese-20220414T080103Z-001.zip /content

In [ ]:
!unzip /content/t5-small-vietnamese-20220414T080103Z-001.zip

Archive:  /content/t5-small-vietnamese-20220414T080103Z-001.zip
  inflating: t5-small-vietnamese/special_tokens_map.json  
  inflating: t5-small-vietnamese/tokenizer.json  
  inflating: t5-small-vietnamese/config.json  
  inflating: t5-small-vietnamese/tokenizer_config.json  
  inflating: t5-small-vietnamese/pytorch_model.bin  


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/t5-small-vietnamese')

In [ ]:
from sklearn.model_selection import train_test_split

train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
# test_data =  Dataset.from_pandas(test_df)

In [ ]:
val_data

Dataset({
    features: ['Unnamed: 0', 'clean_review', 'labels', '__index_level_0__'],
    num_rows: 553
})

In [ ]:
max_input_length = 512
max_target_length = 512


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["clean_review"], max_length=max_input_length, truncation=True
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
# train_data = train_data.select(range(15000))
# val_data = val_data.select(range(1500))

In [ ]:
tokenized_datasets = train_data.map(preprocess_function, batched=True,   remove_columns=["__index_level_0__","clean_review","labels"],)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets_val = val_data.map(preprocess_function, batched=True,   remove_columns=["__index_level_0__","clean_review","labels"],)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets_val

Dataset({
    features: ['Unnamed: 0', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 553
})

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=0632c26899abfe711b5ffb14e05b885f31ddf2e01582268873999c3a7fc53bf2
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
from datasets import load_metric

rouge_score = load_metric("rouge")


Train Model
---



In [ ]:
from transformers import AutoModelForSeq2SeqLM,T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/t5-small-vietnamese')

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
num_train_epochs = 10
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets) // batch_size
# model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir="/content",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_steps=logging_steps,
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

In [ ]:
import numpy as np
from datasets import load_metric
glue_metric = load_metric('glue', 'mrpc')
def compute_metrics(eval_pred):
    
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # results = glue_metric.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2211
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1390


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.265600,0.236427,63.039200,48.025500,61.794400,61.681200
2,0.256600,0.225870,66.556600,51.201400,66.213300,66.288100
3,0.231000,0.212519,68.416300,53.659200,68.013300,68.087400
4,0.212700,0.204533,71.148300,55.863000,70.743100,70.786500
5,0.197200,0.202527,71.728300,56.717500,71.467400,71.484200
6,0.186700,0.200610,72.697600,57.130100,72.045000,72.027700
7,0.174500,0.197264,72.866400,57.399900,72.384900,72.422200
8,0.163200,0.196193,72.992600,57.973400,72.307900,72.287400
9,0.162000,0.199480,73.051000,57.810000,72.608100,72.561800
10,0.155500,0.198436,73.595100,58.268200,73.156700,73.143600


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 553
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 553
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 553
  Batch size = 1

TrainOutput(global_step=1390, training_loss=0.20015822940593142, metrics={'train_runtime': 293.5578, 'train_samples_per_second': 75.317, 'train_steps_per_second': 4.735, 'total_flos': 1269230656094208.0, 'train_loss': 0.20015822940593142, 'epoch': 10.0})

In [ ]:
!zip -r /content/drive/MyDrive/model_ai.zip /content/checkpoint-1000

  adding: content/checkpoint-1000/ (stored 0%)
  adding: content/checkpoint-1000/special_tokens_map.json (deflated 86%)
  adding: content/checkpoint-1000/pytorch_model.bin (deflated 7%)
  adding: content/checkpoint-1000/trainer_state.json (deflated 75%)
  adding: content/checkpoint-1000/tokenizer_config.json (deflated 82%)
  adding: content/checkpoint-1000/optimizer.pt (deflated 7%)
  adding: content/checkpoint-1000/scheduler.pt (deflated 49%)
  adding: content/checkpoint-1000/rng_state.pth (deflated 27%)
  adding: content/checkpoint-1000/config.json (deflated 62%)
  adding: content/checkpoint-1000/tokenizer.json (deflated 74%)
  adding: content/checkpoint-1000/training_args.bin (deflated 48%)



Test model
---



In [ ]:
import torch

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

model = T5ForConditionalGeneration.from_pretrained('/content/checkpoint-1000')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/checkpoint-1000")

Didn't find file /content/checkpoint-1000/spiece.model. We won't load it.
Didn't find file /content/checkpoint-1000/added_tokens.json. We won't load it.
loading file None
loading file /content/checkpoint-1000/tokenizer.json
loading file None
loading file /content/checkpoint-1000/special_tokens_map.json
loading file /content/checkpoint-1000/tokenizer_config.json


In [ ]:

import torch

print("input:")
input_text = input()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)
    print(source_ids)
    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=1028,
        num_beams=4,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=False,
        no_repeat_ngram_size=3
    )
    print(generated_ids)
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

input:
	các món ăn_ở đây không có gì đặc_sắc có_lẽ nấu theo kiểu nhà_hàng cho khách tây nên các món không có j đặc_trưng nhiều món nhưng tầm_thường nêm nếm rất lạt_lẽo không đúng khẩu_vị việt
tensor([[  113,  1239,   383, 20442,   119,   416,   115,   109,   294,   535,
         20442,   266,  5390,   109, 20442,   640,  5130,  2379,   164,  1174,
           142, 20442, 17035,   116,   466,  1314,   283,   113,  1239,   115,
           109,  7010,   535, 20442,  4896,  4996,   186,  1239,   229,  1433,
         20442, 21668, 12314,  7314,   314,  1145,   192, 20442,   640,  5130,
           318,   115,   755,   629, 20442, 16314,  5743,     1]],
       device='cuda:0')
tensor([[    0, 14528, 14714, 20442, 12978,  2858,   169,  8901, 10768,  6393,
           105, 25229,  1621, 20442,   350,  2159,  2858,  9871,  6589,  6393,
             1]], device='cuda:0')

output:
{nha_hang#negative},{an_uong#normal}


In [ ]:
!zip 